In [2]:
!kaggle competitions download -c contradictory-my-dear-watson

# 0 for entailment, 1 for neutral, 2 for contradiction


  0%|          | 0.00/1.78M [00:00<?, ?B/s]
 56%|#####6    | 1.00M/1.78M [00:00<00:00, 7.65MB/s]
100%|##########| 1.78M/1.78M [00:00<00:00, 11.5MB/s]


In [5]:
from zipfile import ZipFile
import polars as pl

with ZipFile('./contradictory-my-dear-watson.zip', 'r') as zObject:
    zObject.extractall(path="./csv")
    
train_df = pl.scan_csv("csv/train.csv").collect()
test_df = pl.scan_csv("csv/test.csv").collect()

In [9]:
train_df

id,premise,hypothesis,lang_abv,language,label
str,str,str,str,str,i64
"""5130fd2cb5""","""and these comm…","""The rules deve…","""en""","""English""",0
"""5b72532a0b""","""These are issu…","""Practice group…","""en""","""English""",2
"""3931fbe82a""","""Des petites ch…","""J'essayais d'a…","""fr""","""French""",0
"""5622f0c60b""","""you know they …","""They can't def…","""en""","""English""",0
"""86aaa48b45""","""ในการเล่นบทบาท…","""เด็กสามารถเห็น…","""th""","""Thai""",1
"""ed7d6a1e62""","""Bir çiftlikte …","""Çiftlikte insa…","""tr""","""Turkish""",0
"""5a0f4908a0""","""ریاست ہائے متح…","""ہیگ کی تفتیش ا…","""ur""","""Urdu""",0
"""fdcd1bd867""","""From Cockpit C…","""From St. Ann's…","""en""","""English""",2
"""7cfb3d272c""","""Look, it's you…","""The boss will …","""en""","""English""",1


# Filtering Arabic Data

### 0. entailed

In [18]:
# arabic, entailed
ar_ent = train_df.filter(
    (pl.col("lang_abv") == 'ar') & 
    (pl.col("label") == 0)
)
ar_ent

id,premise,hypothesis,lang_abv,language,label
str,str,str,str,str,i64
"""91b03f6bf4""","""إذا أمكن ، تعر…","""حاول أن تفهم ا…","""ar""","""Arabic""",0
"""e5c40e04ec""","""ولكن قد تكون …","""بلد قد وصل سري…","""ar""","""Arabic""",0
"""f5ff8d63ad""","""ولكن مع تقدمها…","""لم تعترف أبداً…","""ar""","""Arabic""",0
"""04c794b893""","""انشغل روكفيلير…","""كان روكفلر يعط…","""ar""","""Arabic""",0
"""74465b80fd""","""(dba) ، وهي من…","""يعمل بها متطوع…","""ar""","""Arabic""",0
"""b0d63c8d50""","""ندعوك للمشاركة…","""إذا كنت ترعى ط…","""ar""","""Arabic""",0
"""80317c914a""","""أدرك تماما ما …","""كان عليك أن تف…","""ar""","""Arabic""",0
"""864dc32aa6""","""يتم استخدام ال…","""يخلق الإرهابيو…","""ar""","""Arabic""",0
"""c84c340b30""","""هاه اه اه ليس …","""نمتلك سموات تق…","""ar""","""Arabic""",0


In [19]:
for row in ar_ent.iter_rows():
    print(f" Premise: {row[1]} \n Hypothesis: ({row[2]}) \n\n")

 Premise: إذا أمكن ، تعرّف على المؤامرة مسبقًا. 
 Hypothesis: (حاول أن تفهم الحبكة في البداية، إذا كنت تستطيع.) 


 Premise: ولكن قد  تكون دم العقل الأن . 
 Hypothesis: (بلد قد وصل سريعاً لاستنتاج عن موقفه.) 


 Premise: ولكن مع تقدمها في العمر ، لم تعترف أنها كانت على خطأ ، ولكنها غيرت سلوكها. 
 Hypothesis: (لم تعترف أبداً أنها كانت على خطأ.) 


 Premise: انشغل روكفيلير في هذا العطاء المكروب بينما بدأت تاربيل الملاك المنتقم في تمزيق لحمه في مجلة ماكلورز. 
 Hypothesis: (كان روكفلر يعطي.) 


 Premise: (dba) ، وهي منظمة خيرية ومتطوعين وغير ربحية ، توفر معلومات حرة عن العمل الحر ومساعدة للأعمال التجارية للأشخاص ذوي الإعاقة ، وللمهنيين في مجال إعادة التأهيل المهني والاستشارات المهنية والتجارية. 
 Hypothesis: (يعمل بها متطوعون.) 


 Premise: ندعوك للمشاركة في مستقبل اطفالنا  عن طريق رعاية طفل هندي أمريكي أو الانضمام إلى دائرة العضوية لدعم مشاريع التعليم المجتمعية. 
 Hypothesis: (إذا كنت ترعى طفلاً أمريكياً هندياً، فستشارك في مستقبل الأطفال.) 


 Premise: أدرك تماما ما فعلت، وأتفهم ذلك جزئيا

### 1. neutral

In [20]:
# arabic, neutral
ar_neu = train_df.filter(
    (pl.col("lang_abv") == 'ar') & 
    (pl.col("label") == 1)
)
ar_neu

id,premise,hypothesis,lang_abv,language,label
str,str,str,str,str,i64
"""ee747eb8ca""","""في الواقع، كلم…","""معظم الناس لا …","""ar""","""Arabic""",1
"""0fba15690f""","""هناك نوعان من …","""من المفيد أن ت…","""ar""","""Arabic""",1
"""79cf105ac2""","""وعلى بعد بضعة …","""كان لكل سفينة …","""ar""","""Arabic""",1
"""24e9a5e06d""","""نعم أنا في الح…","""أخواتي أكبر من…","""ar""","""Arabic""",1
"""cf90be5cd9""","""لقد أرسلت لك، …","""الأخبار التي ت…","""ar""","""Arabic""",1
"""cbe9edd4e1""","""الضابط الذي شه…","""انهار البرج ال…","""ar""","""Arabic""",1
"""ca7563d409""","""فرضاً بأنه خلا…","""لا يمكن إجراء …","""ar""","""Arabic""",1
"""1360f6eb78""","""سواء كان عدم ا…","""كانت شرطة نيوي…","""ar""","""Arabic""",1
"""c2b19d6167""","""أنت تعيش وتتعل…","""اختبار الطائرا…","""ar""","""Arabic""",1


In [21]:
for row in ar_neu.iter_rows():
    print(f" Premise: {row[1]} \n Hypothesis: ({row[2]}) \n\n")

 Premise: في الواقع، كلمة كوارك في قاموس أوكسفورد ا كفعل يعني `تشاءم، مع مراجع القرن ال19 تشير إلى الضفادع والغربان، و الملك الحزين. 
 Hypothesis: (معظم الناس لا يعرفون أصل كلمة كوارك وأنها تشير إلى الضوضاء التي يصدرها الضفدع.) 


 Premise: هناك نوعان من المزايا التطورية لمتوسط المظهر. 
 Hypothesis: (من المفيد أن تبدو عاديا لأن الناس يتركونك في حالك.) 


 Premise: وعلى بعد بضعة أميال من هذا الجانب ، بعدها ، وجاءت تسرع ثلاث سفن بيضاء كبيرة. 
 Hypothesis: (كان لكل سفينة مائتي راكب.) 


 Premise: نعم أنا في الحقيقة كان لدي كبار السن من المغنين كبار السن أو كبار السن كبار السن الأخوات المطربين 
 Hypothesis: (أخواتي أكبر مني بكثير.) 


 Premise: لقد أرسلت لك، كابتن بلود، بشأن بعض الأخبار التي وصلتني للتو. 
 Hypothesis: (الأخبار التي تلقيتها صدمتني حتى النخاع.) 


 Premise: الضابط الذي شهد هدم البرج الشمالي، نقل الخبر لوحدات النظام الكهربائي في البرج الشمالي أثناء إعطائه لتوجيهات الإخلاء 
 Hypothesis: (انهار البرج الجنوبي لمدة 30 دقيقة قبل أن يتحدث الضابط إلى وحدات ESU في البرج الشمالي.) 




### 2. contradiction

In [23]:
# arabic, neutral
ar_cont = train_df.filter(
    (pl.col("lang_abv") == 'ar') & 
    (pl.col("label") == 2)
)
ar_cont

id,premise,hypothesis,lang_abv,language,label
str,str,str,str,str,i64
"""cc18ec8d15""","""حسنا، أنا في ا…","""لم يسبق لي أن …","""ar""","""Arabic""",2
"""05ab8a9326""","""لم تصل روح الل…","""اسبانيا لم تكن…","""ar""","""Arabic""",2
"""89467c0148""","""الكلمة الذاتية…","""الرضا الذاتي ل…","""ar""","""Arabic""",2
"""9a88f7c844""","""بالنسبة إلى ال…","""لم تكن أنظمة ا…","""ar""","""Arabic""",2
"""a2fe5baa5c""","""إنهم يطلقون ال…","""يصنعون الكب كي…","""ar""","""Arabic""",2
"""5be385dc12""","""أنا ... أعتقد …","""تحدث بذكاء بحز…","""ar""","""Arabic""",2
"""7ae7b52a62""","""مخبر تينيسي يس…","""وفقا للمخبر ، …","""ar""","""Arabic""",2
"""b0afb96e00""","""كلمتا blood و …","""الغذاء هو بالض…","""ar""","""Arabic""",2
"""1fe6c0e158""","""تضمنت السلسلة …","""لم تكن هناك خط…","""ar""","""Arabic""",2


In [24]:
for row in ar_cont.iter_rows():
    print(f" Premise: {row[1]} \n Hypothesis: ({row[2]}) \n\n")

 Premise: حسنا، أنا في التكساس ولدينا مدرس مات من مرض الإيدز 
 Hypothesis: (لم يسبق لي أن ذهبت إلى تكساس.) 


 Premise: لم تصل روح الليبرالية السائدة في أوروبا إلى إسبانيا إلا في وقت متأخرة. 
 Hypothesis: (اسبانيا لم تكن ابدا ليبرالية.) 


 Premise: الكلمة الذاتية المرضية، والكلمة العادية بدلاً من الكلمة القانونية، هي صانعة للمتاعب وينبغي تجنبها. 
 Hypothesis: (الرضا الذاتي ليس مشكلة.) 


 Premise: بالنسبة إلى الإرسال المتأخر، يرجى مراجعة سجلات الـ FDNY، تقرير الإرسال بمساعدة الكمبيوتر، خانة التنبيه 8087، 11 سبتمبر 2001 ، 09: 03: 00-09: 10: 02. 
 Hypothesis: (لم تكن أنظمة الإرسال بمساعدة الكمبيوتر موجودة في نيويورك حتى عام 2008.) 


 Premise: إنهم يطلقون النار على الطلاب ، أليس كذلك؟ 
 Hypothesis: (يصنعون الكب كيك للطلاب ، أليس كذلك؟) 


 Premise: أنا ... أعتقد أنه قال ، كالفيرلي ، بين الشك وعدم اليقين. 
 Hypothesis: (تحدث بذكاء بحزم واضح، عالماً أن لديه اليقين.) 


 Premise: مخبر تينيسي يستخدم المصطلح dog weather للإشارة إلى الطقس الحار بدون أمطار وهو المصطلح المستمد من dog days للتعب